In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

Using backend: pytorch


In [2]:
NB_NUMBER = 55

In [3]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../../data/dataset/',
                                 #transform=T.ToSparseTensor()
                                 )
dataset

PygNodePropPredDataset()

In [4]:
torch.cuda.is_available()

True

In [5]:
data = dataset[0]
data

Data(edge_index=[2, 1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [6]:
type(data)

torch_geometric.data.data.Data

In [7]:
split_idx = dataset.get_idx_split()
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [8]:
all_edges = data.edge_index

In [9]:
all_edges.shape

torch.Size([2, 1166243])

In [10]:
torch.manual_seed(12345)
idx = torch.randperm(data.edge_index.size(1))
idx

tensor([620901, 244106, 151922,  ..., 667426,  64372, 582625])

In [11]:
n_val_edges = n_test_edges = int(data.edge_index.size(1) * 0.025)
n_val_edges, n_test_edges

(29156, 29156)

In [12]:
val_idx = idx[:n_val_edges]
test_idx = idx[n_val_edges:(n_val_edges + n_test_edges)]
val_idx, test_idx

(tensor([620901, 244106, 151922,  ..., 406803, 294223, 329780]),
 tensor([  72465,  954986,  819672,  ...,  383355,  221124, 1049776]))

In [13]:
val_edges = data.edge_index[:, val_idx]
test_edges = data.edge_index[:, test_idx]
val_edges

tensor([[126442, 142022,  45460,  ...,  16499,  41597,  64196],
        [167681, 104032,  26564,  ...,  76002, 149932, 137760]])

In [14]:
val_edges.size(1)

29156

In [15]:
neg_val_edges = negative_sampling(all_edges, num_nodes=data.x.size(0),
                                  num_neg_samples=val_edges.size(1), method='sparse')
neg_test_edges = negative_sampling(all_edges, num_nodes=data.x.size(0),
                                   num_neg_samples=test_edges.size(1), method='sparse')

In [16]:
remove_idx = torch.cat([val_idx, test_idx])
remove_idx

tensor([ 620901,  244106,  151922,  ...,  383355,  221124, 1049776])

In [17]:
def remove_rows(tensor, indices):
    mask = torch.ones(tensor.size(1), dtype=torch.bool)
    mask[indices] = False
    return tensor[:, mask]

In [18]:
data.edge_index = remove_rows(data.edge_index, remove_idx)
data.edge_index.size()

torch.Size([2, 1107931])

In [19]:
T.ToSparseTensor()(data)
data

Data(adj_t=[169343, 169343, nnz=1107931], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [20]:
data.adj_t = data.adj_t.to_symmetric()

In [21]:
row, col, _ = data.adj_t.coo()
edge_index = torch.stack([col, row], dim=0)
edge_index

tensor([[   411,    640,   1162,  ..., 163274,  27824, 158981],
        [     0,      0,      0,  ..., 169341, 169342, 169342]])

In [22]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, out_channels,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, out_channels))
        for _ in range(num_layers - 1):
            self.convs.append(SAGEConv(out_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, out_channels, bias=True):
        super(LinkPredictor, self).__init__()
        self.lin = torch.nn.Linear(1, out_channels, bias=bias)

    def reset_parameters(self):
        self.lin.weight.data.fill_(1)
        self.lin.bias.data.fill_(0)

    def forward(self, x_i, x_j):
        cos_sim = torch.sum(
            torch.mul(F.normalize(x_i), 
                      F.normalize(x_j)), 
            dim=1,
            keepdim=True
        )
        x = self.lin(cos_sim)
        return torch.sigmoid(x)

In [23]:
device=0
log_steps=1
num_layers=3
hidden_channels=50
dropout=0.5
batch_size=128 * 1024
lr=0.005
epochs=300
eval_steps=5
runs=10

In [24]:
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

In [25]:
adj_t = data.adj_t.to(device)

In [26]:
torch.manual_seed(12345)

In [27]:
model = SAGE(hidden_channels, hidden_channels,
             dropout).to(device)

In [28]:
emb = torch.nn.Embedding(data.num_nodes, hidden_channels).to(device)

In [29]:
predictor = LinkPredictor(1).to(device)

In [30]:
predictor.lin.weight

Parameter containing:
tensor([[-0.8062]], device='cuda:0', requires_grad=True)

In [31]:
emb.weight.size(0), emb.weight.size(1)

(169343, 50)

In [32]:
torch.nn.init.xavier_uniform_(emb.weight)

Parameter containing:
tensor([[ 3.8116e-03, -1.7777e-04,  2.6052e-03,  ..., -9.5538e-04,
         -5.6698e-03,  2.9117e-03],
        [ 4.8286e-03,  5.7555e-03,  4.1475e-03,  ...,  2.2556e-03,
          2.7162e-03,  4.5704e-03],
        [ 1.6921e-03, -1.5087e-03,  3.1802e-03,  ..., -4.9685e-03,
         -5.1306e-04, -9.3337e-04],
        ...,
        [ 4.1685e-03,  3.3565e-03,  3.5653e-03,  ...,  3.1168e-03,
         -1.2510e-03, -3.7040e-03],
        [-6.9713e-04,  2.8990e-03,  4.6939e-03,  ...,  3.2265e-03,
          2.1993e-03,  4.2817e-03],
        [ 4.2867e-03, -3.3813e-03,  3.8860e-05,  ...,  2.8638e-03,
          5.3680e-04, -6.6582e-05]], device='cuda:0', requires_grad=True)

In [33]:
model.reset_parameters()
predictor.reset_parameters()

In [34]:
predictor.lin.weight

Parameter containing:
tensor([[1.]], device='cuda:0', requires_grad=True)

In [35]:
predictor.lin.bias

Parameter containing:
tensor([0.], device='cuda:0', requires_grad=True)

In [36]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(emb.parameters()) +
    list(predictor.parameters()), lr=lr)

In [37]:
def train(model, predictor, x, adj_t, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = edge_index.t().to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='sparse')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

In [38]:
edge = val_edges[:, 0:5]
edge

tensor([[126442, 142022,  45460,   1964,  59023],
        [167681, 104032,  26564,  13902, 105460]])

In [39]:
def test(model, predictor, x, adj_t, batch_size, val_edges, test_edges, neg_val_edges, neg_test_edges):
    model.eval()
    predictor.eval()
    
    h = model(x, adj_t)
    
    val_pos_out = predictor(h[val_edges[0]], h[val_edges[1]])
    val_pos_loss = -torch.log(val_pos_out + 1e-15).mean()
    
    val_neg_out = predictor(h[neg_val_edges[0]], h[neg_val_edges[1]])
    val_neg_loss = -torch.log(1 - val_neg_out + 1e-15).mean()
    
    
    test_pos_out = predictor(h[test_edges[0]], h[test_edges[1]])
    test_pos_loss = -torch.log(test_pos_out + 1e-15).mean()
    
    test_neg_out = predictor(h[neg_test_edges[0]], h[neg_test_edges[1]])
    test_neg_loss = -torch.log(1 - test_neg_out + 1e-15).mean()
    
    return val_pos_loss + val_neg_loss, test_pos_loss + test_neg_loss

In [40]:
def save_models(model, predictor, embedding, epoch):
    torch.save(model, f'../../models/graphsage_link_pred/{NB_NUMBER}_model_epoch{epoch:04d}.pt')
    torch.save(embedding, f'../../models/graphsage_link_pred/{NB_NUMBER}_embedding_epoch{epoch:04d}.pt')
    torch.save(predictor, f'../../models/graphsage_link_pred/{NB_NUMBER}_predictor_epoch{epoch:04d}.pt')

In [41]:
losses = pd.DataFrame(columns=[
    'epoch',
    'train_loss',
    'valid_loss',
    'test_loss'
])

In [42]:
for epoch in range(1, 1 + epochs):
    loss = train(model, predictor, emb.weight, adj_t,
                 optimizer, batch_size)
    
    if epoch % eval_steps == 0:
        valid_loss, test_loss = test(model, predictor, emb.weight, adj_t, batch_size, 
                                     val_edges, test_edges, neg_val_edges, neg_test_edges)
        save_models(model, predictor, emb, epoch)
        print(f'Epoch: {epoch:04d}, '
              f'Train Loss: {loss:.4f}, '
              f'Valid loss: {valid_loss:.4f}, '
              f'Test loss: {test_loss:.4f}')
        losses = losses.append({
            'epoch': epoch,
            'train_loss': loss,
            'valid_loss': valid_loss.item(), 
            'test_loss': test_loss.item()
        }, ignore_index=True)

Epoch: 0005, Train Loss: 0.9849, Valid loss: 0.9955, Test loss: 0.9949
Epoch: 0010, Train Loss: 0.8149, Valid loss: 0.8388, Test loss: 0.8351
Epoch: 0015, Train Loss: 0.7035, Valid loss: 0.7395, Test loss: 0.7332
Epoch: 0020, Train Loss: 0.6226, Valid loss: 0.6679, Test loss: 0.6626
Epoch: 0025, Train Loss: 0.5613, Valid loss: 0.6135, Test loss: 0.6097
Epoch: 0030, Train Loss: 0.5113, Valid loss: 0.5717, Test loss: 0.5660
Epoch: 0035, Train Loss: 0.4712, Valid loss: 0.5374, Test loss: 0.5322
Epoch: 0040, Train Loss: 0.4373, Valid loss: 0.5087, Test loss: 0.5042
Epoch: 0045, Train Loss: 0.4079, Valid loss: 0.4858, Test loss: 0.4816
Epoch: 0050, Train Loss: 0.3829, Valid loss: 0.4645, Test loss: 0.4609
Epoch: 0055, Train Loss: 0.3608, Valid loss: 0.4473, Test loss: 0.4433
Epoch: 0060, Train Loss: 0.3415, Valid loss: 0.4322, Test loss: 0.4285
Epoch: 0065, Train Loss: 0.3249, Valid loss: 0.4205, Test loss: 0.4172
Epoch: 0070, Train Loss: 0.3097, Valid loss: 0.4102, Test loss: 0.4053
Epoch:

In [43]:
losses.to_csv(f'../../data/results/graphsage_link_pred/{NB_NUMBER}_losses.csv', index=False)